# Intro
This notebook shows how we processed American Community Survey data, downloaded from IPUMS, into usable reference tables for our analysis.

In [1]:
import pandas as pd
import numpy as np
import os
from pprint import pprint
from statsmodels.stats.weightstats import DescrStatsW


pd.set_option("max_columns",0)

Data pulled from [IPUMS](https://usa.ipums.org/usa/index.shtml). Data pull is filtered on 2 things: 
1) Person lives in the Northeast Region (Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, Vermont, New Jersey, New York, Pennsylvania)  
2) Person works in New York State  
![ipums screenshot](images/ipums_data_pull.png)

### Read in data, clean up columns

In [2]:
### Read in reference tables. Puma names, TranWork reference table
puma_df = pd.read_csv("ipums_data/pumanames_us_2019_clean.csv",index_col=0)
### Tranwork allows 1-many matches to the HBD TranWork column
tranwork_df = pd.read_csv("ipums_data/ipums_tranwork_reference_updated.csv",index_col=0)

In [3]:
### Read in IPUMS data, join to puma + tranwork reference tables
ipums_df = pd.read_csv("ipums_data/ipums_data_all_NE.csv") 
ipums_df = ipums_df.merge(right=puma_df,on=["STATEFIP","PUMA"])
ipums_df = ipums_df.merge(right=tranwork_df,on=["TRANWORK"])

In [4]:
### Clean up some columns and convert to legible values

## Sex
ipums_df["SEX"] = ipums_df["SEX"].replace({1:"M",2:"F"})

## Health Conditions
health_cols = ['DIFFREM','DIFFPHYS','DIFFMOB','DIFFCARE','DIFFSENS','DIFFEYE','DIFFHEAR']
ipums_df[health_cols]=ipums_df[health_cols].replace({1:"No",2:"Yes"})

## Home Ownership / Rental
ipums_df["OWNERSHP_LABEL"]=ipums_df["OWNERSHP"].replace({0:"N/A",1:"Rent",2:"Own"})

## Race
ipums_df["RACE_LABEL"]=ipums_df["RACE"].replace({1:"White",2:"BlackAA",3:"AmerIndianAlaskan"\
                                                    ,4:"Chinese",5:"Japanese",6:"OtherAsianPacIsl"\
                                                    ,7:"OtherRace",8:"TwoRaces",9:"ThreePlusRaces"})
## Education Level
ipums_df["EDUC_LABEL"]=ipums_df["EDUC"].replace({0:"N/A_NoSchool",1:"Grade0_4",2:"Grades5_8"\
                                                    ,3:"Grade9",4:"Grade10",5:"Grade11"\
                                                    ,6:"Grades12",7:"College_1Year",8:"College_2Year"\
                                                    ,9:"College_3Year",10:"College_4Year"\
                                                 ,11:"College_5PlusYears"})

In [5]:
### Check out the data
print((ipums_df.shape))
ipums_df.head(5)

(289250, 59)


,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,REGION,STATEFIP,COUNTYFIP,DENSITY,CITY,PUMA,STRATA,GQ,OWNERSHP,OWNERSHPD,CINETHH,VEHICLES,PERNUM,PERWT,SEX,AGE,RACE,RACED,HISPAN,HISPAND,EDUC,EDUCD,EMPSTAT,EMPSTATD,OCC,IND,WKSWORK2,UHRSWORK,INCTOT,POVERTY,DIFFREM,DIFFPHYS,DIFFMOB,DIFFCARE,DIFFSENS,DIFFEYE,DIFFHEAR,PWSTATE2,PWCOUNTY,PWPUMA00,TRANWORK,CARPOOL,RIDERS,TRANTIME,DEPARTS,ARRIVES,PUMA_NAME,TRANWORK_DESC,TRANWORK_HBDMATCH,TRANWORK_CLEAN,OWNERSHP_LABEL,RACE_LABEL,EDUC_LABEL
0,2017,201701,241934,2017000002252,55.0,2017002419341,11,9,5,415.4,0,500,50009,1,2,22,1,1,1,55.0,M,44,1,100,0,0,6,63,1,10,6420,770,5,40,20000,158,No,No,No,No,No,No,No,36,0,3100,10,1,1,90,702,834,Litchfield County,"Auto, truck, or van",AutoOccupants,AutoOccupants,Own,White,Grades12
1,2017,201701,242995,2017000107496,37.0,2017002429951,11,9,5,415.4,0,500,50009,1,1,13,1,3,2,33.0,M,60,1,100,0,0,10,101,1,10,9030,6070,6,40,400000,501,No,No,No,No,No,No,No,36,81,4100,10,1,1,90,1135,1304,Litchfield County,"Auto, truck, or van",AutoOccupants,AutoOccupants,Rent,White,College_4Year
2,2017,201701,243613,2017000168483,86.0,2017002436131,11,9,5,415.4,0,500,50009,1,1,13,1,2,2,181.0,F,29,6,660,0,0,6,65,1,10,5620,1770,6,36,19500,217,No,No,No,No,No,No,No,36,0,3100,10,1,1,60,817,919,Litchfield County,"Auto, truck, or van",AutoOccupants,AutoOccupants,Rent,OtherAsianPacIsl,Grades12
3,2017,201701,244242,2017000232422,90.0,2017002442421,11,9,5,415.4,0,500,50009,1,1,12,1,6,2,93.0,M,52,1,100,0,0,10,101,1,10,430,3365,6,40,250000,501,No,No,No,No,No,No,No,36,0,3100,10,1,1,65,802,909,Litchfield County,"Auto, truck, or van",AutoOccupants,AutoOccupants,Rent,White,College_4Year
4,2017,201701,244343,2017000242454,29.0,2017002443431,11,9,5,415.4,0,500,50009,1,1,13,1,2,2,22.0,M,47,1,100,0,0,10,101,1,10,2825,6890,6,55,160000,501,No,No,No,No,No,No,No,36,0,3100,10,1,1,60,702,804,Litchfield County,"Auto, truck, or van",AutoOccupants,AutoOccupants,Rent,White,College_4Year


In [6]:
### Filter down to just people who work in Manhattan (the lowest relevant granularity ACS has for P.O.W.)
### Has to be done at both state and PWPuma level - PWPUMA is state dependent
works_in_ny = ipums_df['PWSTATE2']==36
works_in_manhattan = ipums_df["PWPUMA00"]==3800

ipums_df = ipums_df[works_in_ny&works_in_manhattan]
print((ipums_df.shape))

(67328, 59)


In [7]:
### Create Hourly Columns from Departs/Arrives times
ipums_df["DEPARTS_HOUR"] = ipums_df["DEPARTS"]//100
ipums_df["ARRIVES_HOUR"] = ipums_df["ARRIVES"]//100

In [8]:
### Confirming that the PERWT (weight per person in ACS) sums to approximately the expected number by year
### high 2 millions work in Manhattan (exact numbers vary by source)
ipums_df.groupby(by=["SAMPLE"]).agg({"PERWT":'sum'}) 
### This looks good

,PERWT
SAMPLE,
201701,2621994.0
201801,2622616.0
201901,2671634.0


# Creating reference tables
Now we wiill break apart the larger IPUMS table into smaller & topical reference tables. All will be joined on `YEAR`, `TransMode`, `Hour`. This will also join to the HBD dataset

### 1) Commute Time

In [9]:
def tag_commute_time(row):
    ct = row['TRANTIME']
    if ct <= 20:
        return "0-20 Minutes"
    elif ct <= 40:
        return "21-40 Minutes"
    elif ct <= 60:
        return "41-60 Minutes"
    elif ct <= 80:
        return "61-80 Minutes"
    else:
        return ">80 Minutes"
    
ipums_df["TRANTIME_BUCKET"] = ipums_df.apply(tag_commute_time,axis=1)

total_people_by_tranmode_hour_ct = ipums_df.groupby(by=["TRANWORK_HBDMATCH","ARRIVES_HOUR","YEAR","TRANTIME_BUCKET"]).agg({"PERWT":"sum"}).reset_index()\
.pivot_table(index=["YEAR",'TRANWORK_HBDMATCH',"ARRIVES_HOUR"],columns="TRANTIME_BUCKET",values='PERWT')
#total counts of people by tran mode, arrives hour and commute bucket
total_people_by_tranmode_hour_ct.fillna(0,inplace=True) 
total_people_by_tranmode_hour_ct.reset_index(inplace=True)
total_people_by_tranmode_hour_ct.rename({"TRANWORK_HBDMATCH":"TransMode","ARRIVES_HOUR":"Hour"},axis=1,inplace=True)
total_people_by_tranmode_hour_ct.head(2)

TRANTIME_BUCKET,YEAR,TransMode,Hour,0-20 Minutes,21-40 Minutes,41-60 Minutes,61-80 Minutes,>80 Minutes
0,2017,AutoOccupants,0,104.0,383.0,317.0,0.0,352.0
1,2017,AutoOccupants,1,0.0,120.0,0.0,0.0,268.0


### 2) Age

In [10]:
total_people_by_tranmode_hour_age =\
    ipums_df.groupby(by=["YEAR","TRANWORK_HBDMATCH","ARRIVES_HOUR","AGE"]).agg({"PERWT":"sum"}).reset_index()\
.pivot_table(index=["YEAR",'TRANWORK_HBDMATCH',"ARRIVES_HOUR"],columns="AGE",values='PERWT')
total_people_by_tranmode_hour_age #total by tran mode, arrival hour, age

total_people_by_tranmode_hour_age.columns = [f"Age_{x}" for x in total_people_by_tranmode_hour_age.columns]

total_people_by_tranmode_hour_age = total_people_by_tranmode_hour_age.reset_index()\
.rename({"TRANWORK_HBDMATCH":"TransMode","ARRIVES_HOUR":"Hour"},axis=1)
total_people_by_tranmode_hour_age.fillna(0,inplace=True)
total_people_by_tranmode_hour_age.head(2)


,YEAR,TransMode,Hour,Age_16,Age_17,Age_18,Age_19,Age_20,Age_21,Age_22,Age_23,Age_24,Age_25,Age_26,Age_27,Age_28,Age_29,Age_30,Age_31,Age_32,Age_33,Age_34,Age_35,Age_36,Age_37,Age_38,Age_39,Age_40,Age_41,Age_42,Age_43,Age_44,Age_45,Age_46,Age_47,Age_48,Age_49,Age_50,Age_51,Age_52,...,Age_54,Age_55,Age_56,Age_57,Age_58,Age_59,Age_60,Age_61,Age_62,Age_63,Age_64,Age_65,Age_66,Age_67,Age_68,Age_69,Age_70,Age_71,Age_72,Age_73,Age_74,Age_75,Age_76,Age_77,Age_78,Age_79,Age_80,Age_81,Age_82,Age_83,Age_84,Age_85,Age_86,Age_87,Age_88,Age_89,Age_90,Age_91,Age_94,Age_95
0,2017,AutoOccupants,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.0,0.0,0.0,0.0,0.0,0.0,34.0,104.0,0.0,0.0,0.0,99.0,0.0,0.0,135.0,0.0,256.0,87.0,0.0,0.0,0.0,0.0,161.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017,AutoOccupants,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.0,...,103.0,0.0,0.0,106.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3) Sex

In [11]:
total_people_by_tranmode_hour_sex =\
    ipums_df.groupby(by=["YEAR","TRANWORK_HBDMATCH","ARRIVES_HOUR","SEX"]).agg({"PERWT":"sum"}).reset_index()\
.pivot_table(index=["YEAR",'TRANWORK_HBDMATCH',"ARRIVES_HOUR"],columns="SEX",values='PERWT')

total_people_by_tranmode_hour_sex.columns = [f"Sex_{x}" for x in total_people_by_tranmode_hour_sex.columns]
total_people_by_tranmode_hour_sex #total by tran mode, arrival hour, age
total_people_by_tranmode_hour_sex = total_people_by_tranmode_hour_sex.reset_index()\
            .rename({"TRANWORK_HBDMATCH":"TransMode","ARRIVES_HOUR":"Hour"},axis=1)
total_people_by_tranmode_hour_sex.fillna(0,inplace=True) 
total_people_by_tranmode_hour_sex.head(2)

,YEAR,TransMode,Hour,Sex_F,Sex_M
0,2017,AutoOccupants,0,395.0,761.0
1,2017,AutoOccupants,1,106.0,282.0


### 4) Origin (where people live)

In [12]:
#### PUMA
total_people_by_origin_loc =\
    ipums_df.groupby(by=["YEAR","TRANWORK_HBDMATCH","ARRIVES_HOUR","PUMA","STATEFIP","PUMA_NAME"])\
    .agg({"PERWT":"sum"}).reset_index()\
#322 pumas is too many for pivot table - leaving data in "Long skinny"
total_people_by_origin_loc = total_people_by_origin_loc.reset_index(drop=True)\
            .rename({"TRANWORK_HBDMATCH":"TransMode","ARRIVES_HOUR":"Hour","PERWT":"Estimated_Commuters"},axis=1)
total_people_by_origin_loc.head(2)

,YEAR,TransMode,Hour,PUMA,STATEFIP,PUMA_NAME,Estimated_Commuters
0,2017,AutoOccupants,0,301,34,Bergen County (South Central)--Hackensack & En...,135.0
1,2017,AutoOccupants,0,305,34,"Bergen County (East)--Tenafly, Park Ridge & Cr...",18.0


### 5) Disabilities

In [13]:
total_people_by_health_binaries = ipums_df.groupby(by=["YEAR","TRANWORK_HBDMATCH","ARRIVES_HOUR"]+health_cols)\
    .agg({"PERWT":"sum"}).reset_index()
total_people_by_health_binaries= total_people_by_health_binaries.reset_index(drop=True)\
            .rename({"TRANWORK_HBDMATCH":"TransMode","ARRIVES_HOUR":"Hour","PERWT":"Estimated_Commuters"},axis=1)
total_people_by_health_binaries.head(2)

,YEAR,TransMode,Hour,DIFFREM,DIFFPHYS,DIFFMOB,DIFFCARE,DIFFSENS,DIFFEYE,DIFFHEAR,Estimated_Commuters
0,2017,AutoOccupants,0,No,No,No,No,No,No,No,1034.0
1,2017,AutoOccupants,0,No,No,No,No,Yes,No,Yes,18.0


### 6) Income

In [14]:
### Income binned into deciles using weighted (via PERWT) 2019 incomes
# https://www.statsmodels.org/stable/generated/statsmodels.stats.weightstats.DescrStatsW.html

weighted_income_stats = DescrStatsW(ipums_df[ipums_df["YEAR"]==2019]['INCTOT'],weights=ipums_df[ipums_df["YEAR"]==2019]["PERWT"])
#https://statisticalatlas.com/region/Northeast/Household-Income - Numbers match other sources
income_bucket_limits = weighted_income_stats.quantile([x for x in np.arange(0,1,0.1)],return_pandas=False)

def bucket_incomes(income):
    for list_idx, amt in enumerate(income_bucket_limits):
        if income <= amt:
            return f"income_decile_{list_idx}"
    return f"income_decile_{list_idx+1}"

ipums_df['INCTOT_DECILE']=ipums_df["INCTOT"].apply(bucket_incomes)
total_people_by_tranmode_hour_incomedecile_ct = \
ipums_df.groupby(by=["YEAR","TRANWORK_HBDMATCH","ARRIVES_HOUR","INCTOT_DECILE"])\
    .agg({"PERWT":"sum"})\
.pivot_table(index=["YEAR",'TRANWORK_HBDMATCH',"ARRIVES_HOUR"],columns="INCTOT_DECILE",values='PERWT')


total_people_by_tranmode_hour_incomedecile_ct.fillna(0,inplace=True)
total_people_by_tranmode_hour_incomedecile_ct.reset_index(inplace=True)
total_people_by_tranmode_hour_incomedecile_ct.rename({"TRANWORK_HBDMATCH":"TransMode","ARRIVES_HOUR":"Hour","PERWT":"Estimated_Commuters"}\
                                                     ,axis=1,inplace=True)

total_people_by_tranmode_hour_incomedecile_ct.head(2)

INCTOT_DECILE,YEAR,TransMode,Hour,income_decile_0,income_decile_1,income_decile_10,income_decile_2,income_decile_3,income_decile_4,income_decile_5,income_decile_6,income_decile_7,income_decile_8,income_decile_9
0,2017,AutoOccupants,0,0.0,0.0,0.0,182.0,438.0,61.0,457.0,0.0,0.0,0.0,18.0
1,2017,AutoOccupants,1,0.0,0.0,0.0,0.0,59.0,0.0,226.0,103.0,0.0,0.0,0.0


### 7) Home Ownership

In [15]:
total_people_by_homeownership =\
    ipums_df.groupby(by=["YEAR","TRANWORK_HBDMATCH","ARRIVES_HOUR","OWNERSHP_LABEL","OWNERSHP"])\
    .agg({"PERWT":"sum"}).reset_index()
total_people_by_homeownership = total_people_by_homeownership\
.pivot_table(index=["YEAR",'TRANWORK_HBDMATCH',"ARRIVES_HOUR"],columns="OWNERSHP_LABEL",values='PERWT').fillna(0)
total_people_by_homeownership.columns = [f"HomeStatus_{x}" for x in total_people_by_homeownership.columns]
total_people_by_homeownership.reset_index(inplace=True)
total_people_by_homeownership.rename({"TRANWORK_HBDMATCH":"TransMode","ARRIVES_HOUR":"Hour","PERWT":"Estimated_Commuters"}\
                                                     ,axis=1,inplace=True)
total_people_by_homeownership.head(2)

,YEAR,TransMode,Hour,HomeStatus_N/A,HomeStatus_Own,HomeStatus_Rent
0,2017,AutoOccupants,0,0.0,209.0,947.0
1,2017,AutoOccupants,1,0.0,179.0,209.0


### 8) Race

In [16]:
total_people_by_race =\
    ipums_df.groupby(by=["YEAR","TRANWORK_HBDMATCH","ARRIVES_HOUR","RACE_LABEL","RACE"])\
    .agg({"PERWT":"sum"}).reset_index()
total_people_by_race

total_people_by_race = total_people_by_race\
.pivot_table(index=["YEAR",'TRANWORK_HBDMATCH',"ARRIVES_HOUR"],columns="RACE_LABEL",values='PERWT').fillna(0)
total_people_by_race.columns = [f"Race_{x}" for x in total_people_by_race.columns]
total_people_by_race.reset_index(inplace=True)
total_people_by_race.rename({"TRANWORK_HBDMATCH":"TransMode","ARRIVES_HOUR":"Hour","PERWT":"Estimated_Commuters"}\
                                                     ,axis=1,inplace=True)
total_people_by_race.head(2)

,YEAR,TransMode,Hour,Race_AmerIndianAlaskan,Race_BlackAA,Race_Chinese,Race_Japanese,Race_OtherAsianPacIsl,Race_OtherRace,Race_ThreePlusRaces,Race_TwoRaces,Race_White
0,2017,AutoOccupants,0,0.0,225.0,0.0,0.0,79.0,109.0,0.0,0.0,743.0
1,2017,AutoOccupants,1,0.0,59.0,0.0,0.0,120.0,0.0,0.0,0.0,209.0


### 9) Education

In [17]:
## Education
total_people_by_edu =\
    ipums_df.groupby(by=["YEAR","TRANWORK_HBDMATCH","ARRIVES_HOUR","EDUC_LABEL","EDUC"])\
    .agg({"PERWT":"sum"}).reset_index()
total_people_by_edu


total_people_by_edu = total_people_by_edu\
.pivot_table(index=["YEAR",'TRANWORK_HBDMATCH',"ARRIVES_HOUR"],columns="EDUC_LABEL",values='PERWT').fillna(0)
total_people_by_edu.columns = [f"Educ_{x}" for x in total_people_by_edu.columns]
total_people_by_edu.reset_index(inplace=True)
total_people_by_edu.rename({"TRANWORK_HBDMATCH":"TransMode","ARRIVES_HOUR":"Hour","PERWT":"Estimated_Commuters"}\
                                                     ,axis=1,inplace=True)
total_people_by_edu.head(2)

,YEAR,TransMode,Hour,Educ_College_1Year,Educ_College_2Year,Educ_College_4Year,Educ_College_5PlusYears,Educ_Grade0_4,Educ_Grade10,Educ_Grade11,Educ_Grade9,Educ_Grades12,Educ_Grades5_8,Educ_N/A_NoSchool
0,2017,AutoOccupants,0,191.0,161.0,18.0,176.0,0.0,0.0,0.0,0.0,610.0,0.0,0.0
1,2017,AutoOccupants,1,0.0,0.0,120.0,0.0,0.0,103.0,0.0,0.0,165.0,0.0,0.0


### Save all to csv + move to Google Drive database folder

These dataframes are all dropped to a local folder, and then moved to our team repository on Google Drive. Ultimately, the data connects like this:

![db_diagram](images/db_diagram.png)

In [18]:
dir_drop="ipums_data_drop"
os.makedirs(dir_drop,exist_ok=True)

## SEX
total_people_by_tranmode_hour_sex.to_csv(f"{dir_drop}/commuter_sex_counts.csv")

## AGE
total_people_by_tranmode_hour_age.to_csv(f"{dir_drop}/commuter_age_counts.csv")

## ORIGIN PUMA
total_people_by_origin_loc.to_csv(f"{dir_drop}/commuter_origin_counts.csv")

## HEALTH DISABILITIES
total_people_by_health_binaries.to_csv(f"{dir_drop}/commuter_health_disabilities_counts.csv")

## COMMUTE TIME BUCKETS
total_people_by_tranmode_hour_ct.to_csv(f"{dir_drop}/commuter_commute_time_buckets_counts.csv")

## TOTAL INCOME BUCKETS
total_people_by_tranmode_hour_incomedecile_ct.to_csv(f"{dir_drop}/commuter_income_buckets_counts.csv")

## HOME OWNERSHIP
total_people_by_homeownership.to_csv(f"{dir_drop}/commuter_home_ownership_counts.csv")

## RACE
total_people_by_race.to_csv(f"{dir_drop}/commuter_race_counts.csv")

## EDUC
total_people_by_edu.to_csv(f"{dir_drop}/commuter_educ_counts.csv")
